# Generative Supervised Fine-tuning of GPT-2

Now that we have our GPT-2 model all trained up - we need a way we can get it to generate what we want.

In the following notebook, we're going to use an approach called "Supervised Fine-tuning" to achieve our goals today.

In essence, we're going to use each example as a self-contained unit (with potential for something called "packing") and this is going to allow us to build "labeled" data.

For this notebook, we're going to be flying quite high up in the levels of abstraction. Take extra care to look into the libraries we're using today!

Let's start by grabbing our dependencies, as always:



In [ ]:
!pip install transformers accelerate datasets trl bitsandbytes -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Dataset Curation

We're going to be fine-tuning our model on SQL generation today.

First thing we'll need is a dataset to train on!

We'll use [this](https://huggingface.co/datasets/b-mc2/sql-create-context) dataset today!

First up, let's load it and take a look at what we've got.

In [ ]:
from datasets import load_dataset

sql_dataset = load_dataset("b-mc2/sql-create-context")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [ ]:
sql_dataset

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 78577
    })
})

In [ ]:
sql_dataset['train'][0]

{'answer': 'SELECT COUNT(*) FROM head WHERE age > 56',
 'question': 'How many heads of the departments are older than 56 ?',
 'context': 'CREATE TABLE head (age INTEGER)'}

So, we've got ~78.5K rows of:

- question - a natural language query about
- context - the `CREATE TABLE` statement - which gives us important context about the table
- answer - a SQL query that is aligned with both the question and the context.

Let's split our data into `train`, `val`, and `test` datasets.

We can use our `train` and `val` sets to train and evaluate our model during training - and our `test` set to ultimately benchmark the generations of our model!

In [ ]:
sql_dataset_train_test = sql_dataset["train"].train_test_split(test_size=0.2)

In [ ]:
sql_dataset_train_test

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 62861
    })
    test: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 15716
    })
})

In [ ]:
sql_dataset_val_test = sql_dataset_train_test["test"].train_test_split(test_size=0.5)

In [ ]:
sql_dataset_val_test

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 7858
    })
    test: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 7858
    })
})

In [ ]:
from datasets import DatasetDict

split_sql_dataset = DatasetDict({
    "train" : sql_dataset_train_test["train"],
    "val" : sql_dataset_val_test["train"],
    "test" : sql_dataset_val_test["test"]
})

In [ ]:
split_sql_dataset

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 62861
    })
    val: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 7858
    })
    test: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 7858
    })
})

### Creating a "Prompt"

Now we need to create a prompt that's going to allow us to interact with our model when we desired the trained behaviour.

Think of this as a pattern that aligns the model with our desired outputs.

We need a single text prompt, as that is what the `SFTTrainer` we're going to use to fine-tune our model expects.

The basic idea is that we're going to merge the `question`, `context`, and `answer` into a single block of text that shows the model our desired outputs.

Let's look at what that block needs to look like:

```
{bos_token}### Instruction:
{system_message}

### Input:
{input}

### Context:
{context}

### Response:
{response}{eos_token}
```

Let's look at that from a completed prompt perspective to get a bit more information:

```
<|startoftext|>### Instruction:
You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.
You must output the SQL query that answers the question.

### Input:
How many locations did the team play at on week 7?

### Context:
CREATE TABLE table_24123547_2 (location VARCHAR, week VARCHAR)

### Response:SELECT COUNT(location) FROM table_24123547_2 WHERE week = 7<|endoftext|>
```

As you can see, our prompt contains completed examples of our task. We're going to show our model many of these examples over and over again to teach it to produce outputs that are aligned with our goals!

First step, let's create a template we can use to call `.format()` on while constructing our prompts.

###🏗️Activity:

Create the following templates:

- `TEXT2SQL_TRAINING_PROMPT_TEMPLATE`
- `TEXT2SQL_INFERENCE_PROMPT_TEMPLATE`

> HINT: Remember that during inference we do not want to prepopulate the response.

In [ ]:
TEXT2SQL_TRAINING_PROMPT_TEMPLATE = """\
{bos_token}### Instruction:
{system_message}

### Input:
{input}

### Context:
{context}

### Response:
{response}{eos_token}
"""

TEXT2SQL_INFERENCE_PROMPT_TEMPLATE = """\
{bos_token}### Instruction:
{system_message}

### Input:
{input}

### Context:
{context}
"""

Now let's create a function we can map over our dataset to create the full prompt text block.

###🏗️Activity:

Define a `SYSTEM_MESSAGE` to use with your training data.

In [ ]:
SYSTEM_MESSAGE = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. You must output the SQL query that answers the question.
                 """


In [ ]:
def create_sql_prompt(sample):
  full_prompt = TEXT2SQL_TRAINING_PROMPT_TEMPLATE.format(
      bos_token = "<|startoftext|>",
      eos_token = "<|endoftext|>",
      system_message = SYSTEM_MESSAGE,
      input = sample["question"],
      context = sample["context"],
      response = sample["answer"]
  )

  return {"text" : full_prompt}

I've created this helper-function to be able to see how our model is doing visibly, rather than only through metrics.

In [ ]:
def create_sql_prompt_and_response(sample):
  full_prompt = TEXT2SQL_INFERENCE_PROMPT_TEMPLATE.format(
      bos_token = "<|startoftext|>",
      system_message = SYSTEM_MESSAGE,
      input = sample["question"],
      context = sample["context"]
  )

  ground_truth = sample["answer"]

  return {"full_prompt" : full_prompt, "ground_truth" : ground_truth}

Let's look at an example of a formatted prompt.

In [ ]:
create_sql_prompt(split_sql_dataset["train"][0])

{'text': '<|startoftext|>### Instruction:\nYou are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. You must output the SQL query that answers the question.\n                 \n\n### Input:\nWhere were less than 72 goals made and more than 40 assists made?\n\n### Context:\nCREATE TABLE table_name_74 (venue VARCHAR, goals VARCHAR, assists VARCHAR)\n\n### Response:\nSELECT venue FROM table_name_74 WHERE goals < 72 AND assists > 40<|endoftext|>\n'}

Great!

Now we can map this over our dataset!

In [ ]:
split_sql_dataset = split_sql_dataset.map(create_sql_prompt)

Map:   0%|          | 0/62861 [00:00<?, ? examples/s]

Map:   0%|          | 0/7858 [00:00<?, ? examples/s]

Map:   0%|          | 0/7858 [00:00<?, ? examples/s]

## Load the Model And Preproccessing

Now for the moment we've all been waiting for...

Loading our model!

Let's use the `AutoModelForCausalLM` and `AutoTokenzier` classes from `transformers` to see just how easy this is.

- [`AutoModelForCausalLM`](https://huggingface.co/docs/transformers/v4.35.0/en/model_doc/auto#transformers.AutoModelForCausalLM)
- [`AutoTokenizer`](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoTokenizer)
- [GPT-2 Model Card](https://huggingface.co/gpt2)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "gpt2"

gpt2_base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto"
)

gpt2_tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

We need to make sure our tokenizer has a `pad_token` in order to be able to pad sequences so they're all the same length.

We'll use a little trick here to set our padding token to our eos (end of sequence) token to make training go a little smoother.

In [ ]:
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

We also need to make sure we resize our model to be aligned with the token embeddings. If we didn't do this - we'd face a shape error while training!

In [ ]:
gpt2_base_model.resize_token_embeddings(len(gpt2_tokenizer))

Embedding(50257, 768)

Now let's use the Hugging Face `pipeline` to see what generation looks like for our untrained model.

In [ ]:
from transformers import pipeline, set_seed, GenerationConfig

generator = pipeline('text-generation', model=gpt2_base_model, tokenizer=gpt2_tokenizer)
set_seed(42)

def generate_sample(sample):
  prompt_package = create_sql_prompt_and_response(sample)

  generation_config = GenerationConfig(
      max_new_tokens=50,
      do_sample=True,
      top_k=50,
      temperature=1e-4,
      eos_token_id=gpt2_base_model.config.eos_token_id,
  )

  print("Input processed : ",prompt_package["full_prompt"])

  generation = generator(prompt_package["full_prompt"], generation_config=generation_config)
  print("---------------")
  print("Model Response:")
  print(generation[0]["generated_text"].replace(prompt_package["full_prompt"], ""))
  print("+++++++++++++++")
  print("Ground Truth")
  print(prompt_package["ground_truth"])

Device set to use cuda:0


In [ ]:
generate_sample(split_sql_dataset["test"][0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Model Response:

### Output:

SELECT * FROM table_name_39 WHERE team_id = 0 AND team_id = 1 AND team_id = 2 AND team_id = 3 AND team_id = 4 AND team_id = 5
+++++++++++++++
Ground Truth
SELECT SUM(poles) FROM table_name_39 WHERE class = "125cc" AND team = "matteoni racing team" AND points > 3


## Training the Model

Now that we have our model set up, our tokenizer set up, we can finally begin training!

We'll be using the `TrainingArguments` from Hugging Face's `transformers` library to help us keep track of our hyper-parameters. More information and documentation available:

- [`TrainingArguments`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments)

Let's look at our Trainer, and set some hyper-parameters:

- `per_device_train_batch_size` - this is a batch size that accomodates distributed training
- `gradient_accumulation_steps` - this is exactly the same as the previous notebook, it's a way to "simulate" a large batch size by collecting losses over multiple iterations - scaling them - and then combining them together.
- `gradient_checkpointing` - I'll let the authors speak for themselves [here](https://github.com/cybertronai/gradient-checkpointing). In essence: This saves memory at the cost of computational time.
- `max_grad_norm` - this is the value used for gradient clipping, which is a method of reducing vanishing gradient potential
- `max_steps` - how many steps will we train for?
- `learning_rate` - how fast should we learn?
- `save_total_limit` - how many versions of the model will we save?
- `logging_steps` - how often we should log
- `output_dir` - where to save our checkpoints
- `optim` - which optimizer to use, you'll notice we're using a full precision paged optimizer - this is a performative and stable optimizer - but it uses extra memory
- `lr_scheduler_type` - we are once again using a cosine scheduler!
- `evaluation_strategy` - we have an evaluation dataset, this defines when we should leverage it during training
- `eval_steps` - how many steps we should evaluate for
- `warmup_ration` - how many "warmup" steps we take to reach our full learning rate before we start decaying. this is a ration of our max_steps

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer, SFTConfig

# training_args = TrainingArguments(
#  per_device_train_batch_size=4,
#  gradient_accumulation_steps=4,
#  gradient_checkpointing =True,
#  max_grad_norm= 0.3,
#  ###num_train_epochs=2,
#  max_steps=500,
#  learning_rate=2e-4,
#  save_total_limit=3,
#  logging_steps=10,
#  output_dir="sql_gpt2",
#  optim="paged_adamw_32bit",
#  lr_scheduler_type="cosine",
#  eval_strategy="steps",
#  eval_steps=50,
#  warmup_ratio=0.05,
#  dataset_text_field="text",
#  max_seq_length=1024
# )

from trl import SFTTrainer, SFTConfig

training_args = SFTConfig(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    max_grad_norm=0.3,
    max_steps=500,
    learning_rate=2e-4,
    save_total_limit=3,
    logging_steps=10,
    output_dir="sql_gpt2",
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    eval_strategy="steps",  # correct name
    eval_steps=50,
    warmup_ratio=0.05,
    dataset_text_field="text",  # only works here, not in TrainingArguments
)




###❓Question

Is this process using usupervised, or supervised learning?

\#\#\# YOUR RESPONSE HERE

Now, for our `SFTTrainer` AKA "Where the magic happens".

You can read all about the `SFTTrainer` here:

- [`SFTTrainer`](https://huggingface.co/docs/trl/sft_trainer#trl.SFTTrainer)

This `SFTTrainer` is going to take our above training arguments, our data, our model and our tokenizer, and train it all for us!

Notice that we're setting `max_seq_length` to the maximum context window of our model - this ensures we do not exceed our maximum context window, and will pad our examples up to the maximum context window!

In [ ]:
SFTTrainer?

In [ ]:
trainer = SFTTrainer(
    model=gpt2_base_model,
    args=training_args,
    train_dataset=split_sql_dataset["train"],
    eval_dataset=split_sql_dataset["val"],
    #tokenizer=gpt2_tokenizer
)

In [ ]:
# config = SFTConfig(
#     dataset_text_field="text",
#     max_seq_length=1024,
#     # add other relevant settings
# )
# trainer = SFTTrainer(
#  gpt2_base_model,
#  args=config,
#  train_dataset=split_sql_dataset["train"],
#  eval_dataset=split_sql_dataset["val"],
#  tokenizer=gpt2_tokenizer,
#  args=training_args
# )

# from trl import SFTConfig, SFTTrainer

# config = SFTConfig(
#     dataset_text_field="text",
#     max_seq_length=512,
#     # add other relevant settings
# )

# trainer = SFTTrainer(
#     model=model,
#     args=config,
#     train_dataset=split_sql_dataset["train"],
#     eval_dataset=split_sql_dataset["val"],
#     tokenizer=tokenizer,              # if needed
#     data_collator=data_collator,      # if needed
#     # other settings as needed
# )

TypeError: SFTTrainer.__init__() got an unexpected keyword argument 'dataset_text_field'

###❓Question

What do we use to determine loss in fine-tuning GPT-2?

> HINT: [This](https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt2/modeling_gpt2.py) can help you if you get stuck!

Finally, we can call our `.train()` method and watch it go!

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nursnaaz (nursnaaz-inceptez) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
50,0.640600,0.620792
100,0.590000,0.550932
150,0.591000,0.521895
200,0.514300,0.507554
250,0.532900,0.493912
300,0.489700,0.483951
350,0.465100,0.476043
400,0.489700,0.471360
450,0.485100,0.468904
500,0.474700,0.468597


TrainOutput(global_step=500, training_loss=0.5961755743026733, metrics={'train_runtime': 499.4781, 'train_samples_per_second': 16.017, 'train_steps_per_second': 1.001, 'total_flos': 724936370688000.0, 'train_loss': 0.5961755743026733})

Let's save our fine-tuned model!

In [ ]:
trainer.save_model()

## Testing our Model

Now that we have a fine-tuned model, let's see how it did

In [ ]:
ft_gpt2_model = AutoModelForCausalLM.from_pretrained("sql_gpt2")

In [ ]:
generator = pipeline('text-generation', model=ft_gpt2_model, tokenizer=gpt2_tokenizer, )

Device set to use cuda:0


In [ ]:
split_sql_dataset["test"][0]

{'answer': 'SELECT SUM(poles) FROM table_name_39 WHERE class = "125cc" AND team = "matteoni racing team" AND points > 3',
 'question': 'How many Poles have a Class of 125cc, and a Team of matteoni racing team, and Points larger than 3?',
 'context': 'CREATE TABLE table_name_39 (poles INTEGER, points VARCHAR, class VARCHAR, team VARCHAR)',
 'text': '<|startoftext|>### Instruction:\nYou are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. You must output the SQL query that answers the question.\n                 \n\n### Input:\nHow many Poles have a Class of 125cc, and a Team of matteoni racing team, and Points larger than 3?\n\n### Context:\nCREATE TABLE table_name_39 (poles INTEGER, points VARCHAR, class VARCHAR, team VARCHAR)\n\n### Response:\nSELECT SUM(poles) FROM table_name_39 WHERE class = "125cc" AND team = "matteoni racing team" AND points > 3<|endoftext|>\n'}

In [ ]:
generate_sample(split_sql_dataset["test"][4])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input processed :  <|startoftext|>### Instruction:
You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. You must output the SQL query that answers the question.
                 

### Input:
Which Lost has a Position larger than 2, a Name of sg münchen (n), and a Drawn larger than 0?

### Context:
CREATE TABLE table_name_48 (lost VARCHAR, drawn VARCHAR, position VARCHAR, name VARCHAR)

---------------
Model Response:

### Response:
SELECT lost FROM table_name_48 WHERE position > 2 AND name = "sg münchen (n)" AND drawn > 0
+++++++++++++++
Ground Truth
SELECT COUNT(lost) FROM table_name_48 WHERE position > 2 AND name = "sg münchen (n)" AND drawn > 0


That is *significantly* better.

#### ###❓Question

How might you evaluate your generated SQL? Please provide 3 different methods.

In [ ]:
https://huggingface.co/evaluate-metric/spaces?p=1